In [17]:
from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual
import os
import shutil
import platform
import ipywidgets as ipyw
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import IPython
import warnings
warnings.filterwarnings("ignore")
import json
from json import dumps
import numpy as np
import pandas as pd
import pyemu


In [23]:
base_pst_file = "restart.pst"
base_dir = os.path.join("ouu","base")
scen_pst_file = "scenario.pst"
scen_dir = os.path.join("ouu","scenario")
exe_name = "pestpp-opt"
dv_group = "k1"

if "linux" in platform.platform().lower():
    bin_dir = os.path.join("ouu","bin","linux")
    ext = ''
elif "darwin" in platform.platform().lower():
    bin_dir = os.path.join("ouu","bin","mac")
    ext = ''
else:
    bin_dir = os.path.join("ouu","bin","win")
    ext = ".exe"
    
def load_pst():
    pst = pyemu.Pst(os.path.join(base_dir,base_pst_file))
    pst.control_data.noptmax = 1
    pst.pestpp_options = {}
    pst.pestpp_options["opt_dec_var_groups"] = dv_group
    pst.pestpp_options["base_jacobian"] = "restart.jcb"
    pst.pestpp_options["hotstart_resfile"] = "restart.rei"
    pst.pestpp_options["opt_skip_final"] = True
    pst.pestpp_options["opt_std_weights"] = True
    pst.pestpp_options["opt_direction"] = "max"
    return pst

In [24]:
def scrape_recfile(recfile):
    infeas = False
    with open(recfile,'r') as f:
        for line in f:
            if "best objective function value" in line:
                phi = float(line.strip().split(':')[-1])
                #break

            if "warning: primal solution infeasible" in line:
                infeas = True
    return infeas, phi

In [25]:
def run_pestpp_opt(const_dict,risk=0.5):
    if os.path.exists(scen_dir):
        shutil.rmtree(scen_dir)
    shutil.copytree(base_dir,scen_dir)
    shutil.copy2(os.path.join(bin_dir,exe_name+ext),os.path.join(scen_dir,exe_name+ext))
    pst_scen = load_pst()
    pst_scen.pestpp_options["opt_risk"] = risk
    obs = pst_scen.observation_data
    for const_name,const_percent_change in const_dict.items():
        const_name = const_name.lower()
        assert const_name in obs.obsnme
        obs.loc[const_name,"obsval"] *= const_percent_change
    pst_scen.write(os.path.join(scen_dir,scen_pst_file))
    pyemu.os_utils.run("{0} {1}".format(exe_name,scen_pst_file),cwd=scen_dir)
    infeas,phi = scrape_recfile(os.path.join(scen_dir,scen_pst_file.replace(".pst",".rec")))
    
    return infeas,phi
    

In [26]:
run_pestpp_opt({})

noptmax: 1


(False, 20138.7)

In [22]:
ib = np.loadtxt(os.path.join(base_dir,"ibound_layer_1.ref"))

In [7]:
pst = load_pst()

In [8]:
const_names = [name for name in pst.nnz_obs_names if pst.observation_data.loc[name,"obgnme"].startswith("less_")]

In [9]:
const_names

['sfrc30_1_07300.00',
 'ucn_00_008_015_001',
 'ucn_00_010_012_001',
 'ucn_00_019_013_001',
 'ucn_00_025_009_001',
 'ucn_00_028_005_001',
 'ucn_00_033_011_001']

In [10]:
const_dict = {cn:1.2 for cn in const_names}
run_pestpp_opt(const_dict,risk=0.1)

noptmax: 1


(False, 3.19744e-14)